# Battery Lifetime Calculator

## How to use this document

If you are viewing this on github.com, this document is read only.  Download it and open it
using Jupyter Notebook if you want to edit it.

When viewing in Jupyter, click the run button on each block to step through the calculations.   

Some of the blocks have values that you can change before you click run.

## What does this document do
This calculation turns battery capacity around, and asks, you have a device that does some job, how many jobs can you do?.   

Each job uses high power, but the device also uses a smaller amount of power all the time (its *standby consumption*).

How many jobs can you do in a day, before you need to recharge?

Battery capacity is measured in Ampere-Hours, that is Amps multiplied by Hours.   Its a literal trade-off---draw a lot of amps, you only get a few hours; draw a fraction of an amp, and you get lots of hours.  A 10Ah battery can theoretically deliver 10 Amps for 1 hour, one Amp for 10 hours, or 500mA for 20 hours.   In reality, the relationship might not be so *linear*.

The rated capacity from the manufacturer is usually for a 20 hour discharge, so if you are going to run the battery for a shorter or longer time, you may experience a different effective capacity.    Also, resellers lie outrageously about capacity.

If your load is constant (eg you draw 200mA all the time, then a theoretical lifetime is simply `capacity / current`, eg `10Ah / 200mA => 50 hours`.

Since we are considering a 24 hour discharge, we are close to the manufacturer figure, but practically 
we are using different rates, our gadget will be in standby (sleep mode) when possible and only consume our **peak draw** when doing a job.  The rest of the time we consume a far smaller **standby draw**

We'll use Python and the *Pint* unit library to help us model this scenario.

In [60]:
# Load some library modules.   You can ignore this block, just click run.
import pint
units = pint.UnitRegistry()
from math import floor

## Enter your battery capacity

The rated capacity presumes you use all the power in the battery.   Running many kinds of battery dead flat can damage them, so lets choose a safety factor.   We'll say we only run the battery 80% flat.



In [101]:
# Edit the values in this block below and then click "Run".

number_of_cells = 3

rated_capacity = 120 * units.Ah

depth_of_discharge = 0.5

# Don't change anything below here in this block
nominal_voltage = 3.7 * number_of_cells * units.volt
capacity = rated_capacity * depth_of_discharge
energy_budget = (capacity * nominal_voltage).to("watt_hour");
print("If we are nice to our {:.3} battery we can use up to {} of its rated {} capacity.".format(nominal_voltage, capacity,rated_capacity))
print("That means we have a daily energy budget of {:.4}.".format(energy_budget))

If we are nice to our 11.1 volt battery we can use up to 60.0 ampere_hour of its rated 120 ampere_hour capacity.
That means we have a daily energy budget of 666.0 watt_hour.


## Describe your power needs

### Enter your job power consumption   

How much power do you need when doing a job?


In [92]:
# Edit the values in this block below and then click "Run".

job_power = 20 * units.watt

job_duration = 15 * units.minute

# Don't change anything below here in this block
job_energy_use = (job_power * job_duration).to(units.Wh);
print("At {} for {} we use {} per job".format(job_draw, job_duration, job_energy_use))

At 1500 milliampere for 15 minute we use 5.0 watt_hour per job


### Enter your standby consumption   

How much power do you consume when in standby mode?

Let's take a shortcut and presume we use the standby energy even during jobs.  If we didn't take this shortcut we would need to solve a set of simultaneous equations.


In [103]:

standby_power = 1 * units.watt

# Don't change anything below here in this block
standby_energy_use = (standby_power * 24 * units.hour).to(units.Wh)
print("At {0} standby, our gadget is going to consume {1:.5} each day even when idle."
      .format(standby_power,standby_energy_use))
print("That's {}% of our energy budget just for standby."
      .format(floor(100*standby_energy_use/energy_budget)))

At 1 watt standby, our gadget is going to consume 24.0 watt_hour each day even when idle.
That's 3% of our effective capacity just for standby.


## Caculate the theoretical job capacity

Now we know how much energy we use per day just when idle, we can divide the remainder by 24h to work out how many jobs we could do per day in the best case.    

But we don't work 24 hours.   We have a "working" day, so lets work out what that practically means for service capacity.



In [105]:
# Edit the values in this block below and then click "Run".

working_hours_per_day = 10 * units.hour

# You don't need to edit this block, just click "Run" to see the calculated value
job_energy_budget = energy_budget - standby_energy_use
job_capacity = floor(job_energy_budget / job_energy_use)
print("In theory, we can do up to {} x {} jobs, per day.".format(job_capacity,job_duration))
user_capacity = floor(job_capacity * job_duration / working_hours_per_day)
print("That means that in a {} working day, with no breaks, we could serve {} users at a time".format(working_hours_per_day,user_capacity))

In theory, we can do up to 128 x 15 minute jobs, per day.
That means that in a 10 hour working day, we could serve 3 users at a time


# Calculate a realistic job capacity

Nobody works 100% of the time.   Let's presume we get half a job to rest between each job.

In [109]:
practical_work_hours = working_hours_per_day * 0.75
practical_user_capacity = floor(job_capacity * job_duration / practical_work_hours)
print("With a bit of downtime between jobs, doing only {} a day, we could serve {} users at a time"
.format(practical_work_hours,practical_user_capacity))

With a bit of downtime between jobs, doing only 7.5 hour a day, we could serve 4 users at a time


**Note**: In theory, theory and practice are the same thing.  In practice, they are not.

In reality, there is efficiency, non-linearity and self-discharge to consider, so this figure may not be precise.